In [1]:
using BooleanSatisfiability
using LinearAlgebra, Plots

## Scheduling task:
We have n people's availabilities for 9a, 10a, 11a, 12, 1p, 2p, 3p, 4p defined as a vector of 0 and 1's
We would like to schedule $J$ meetings between different groups of people, represented by $J$ index sets $\mathcal{I_j}\subseteq\{1,\dots,n\}$.

Rules:
* Each meeting $\mathcal{I}_j$ must occur at one time $t$
* All people attending meeting $\mathcal{I}_j$ must be available at time $t$
* All people attending meeting $\mathcal{I}_j$ must not be attending another meeting at time $t$
* No attendee should have >2 hours of consecutive meetings.

In [78]:
n = 5 # number of people
T = 8 # number of times
# nx8 matrix: each row is one attendee's meeting availability
A_bar = [
    1 0 1 0 1 1 1 1
    0 1 1 0 0 1 0 1
    0 1 0 0 0 0 0 1
    1 1 0 1 0 1 0 0
    0 0 1 1 1 1 0 1
]
index_sets = [[1,2,3], [4,5], [1,3,5], [3,4], [1,4]]
J = length(index_sets) # number of meetings

5

Define $\bar a_{it}\in\{0,1\}$ = the known availability of person $i$ at time $t$ and $\bar A\in\{0,1\}^{n\times T}$.

Define $a_{it} \in\{0,1\}$ = 1 if person $i$ is attending a meeting at time $t$, 0 otherwise. $A\in\{0,1\}^{n\times T}$.

Define $m_j\in\{0,1\}^T$ = the time at which meeting $\mathcal{I}_j$ is scheduled, in one-hot encoding.

Since $\bar A$ is known, the goal is to find a satisfying assignment of $A$ and $m_1,\dots,m_J$ such that all meetings can occur following the given rules.

* $m_j$ is the time when meeting $j$ can occur:
$$m_{j,t} = \bigwedge_{i\in\mathcal{I}_j} a_{it}$$

* All people attending $\mathcal{I}_j$ must be available at time $t$ and not attending another meeting:
$$\bigwedge_{it} \bar a_{it} \implies \neg a_{it}$$
$$\bigwedge_j \bigvee_t \left(m_{j,t}\wedge \left(\bigwedge_{j'\neq j} \neg m_{j', t}\right)\right)$$

* No attendee should have more than two consecutive hours of meetings
$$\bigwedge_i \bigwedge_{t=1,\dots,T-2} \neg(a_{i,t}\wedge a_{i,t+1} \wedge a_{i,t+2})$$

In [79]:
A = Bool(n,T,"A")
#past_availability = all(¬A_bar .⟹ ¬A)
booked = BoolExpr[]
for i=1:n
    for t=1:T
        if A_bar[i,t]==0
            push!(booked, ¬A[i,t])
        end
    end
end
unavailability = all(booked)

M = [and(A[index_sets[j], t]) for j=1:J, t=1:T]

# everyone must be available at some time t and not attending another meeting
inner = [and([M[j:j,t]; A[index_sets[j],t]]) for j=1:J, t=1:T]
meetings = [or(inner[j,:]) for j=1:J]

# nobody should have more than 2 consecutive hours of meetings
time_limit = all([¬and(A[i,t:t+2]) for i=1:n, t=1:T-2])

# solve
exprs = [ and(meetings), unavailability, time_limit]
status = sat!(exprs)

:SAT

In [80]:
value(A)

5×8 Matrix{Bool}:
 1  0  0  0  0  1  0  1
 0  0  0  0  0  0  0  1
 0  1  0  0  0  0  0  1
 1  1  0  1  0  1  0  0
 0  0  0  1  0  1  0  1

In [81]:
value(M)

5×8 Matrix{Bool}:
 0  0  0  0  0  0  0  1
 0  0  0  1  0  1  0  0
 0  0  0  0  0  0  0  1
 0  1  0  0  0  0  0  0
 1  0  0  0  0  1  0  0

## Now let's see what this would look like in SMTLIB2

In [82]:
for (i,line) in enumerate(readlines(open("out.smt", "r")))
    println(i, " ", line)
end

1 (declare-const A_1_1 Bool)
2 (declare-const A_2_1 Bool)
3 (declare-const A_3_1 Bool)
4 (declare-const A_1_2 Bool)
5 (declare-const A_2_2 Bool)
6 (declare-const A_3_2 Bool)
7 (declare-const A_1_3 Bool)
8 (declare-const A_2_3 Bool)
9 (declare-const A_3_3 Bool)
10 (declare-const A_1_4 Bool)
11 (declare-const A_2_4 Bool)
12 (declare-const A_3_4 Bool)
13 (declare-const A_1_5 Bool)
14 (declare-const A_2_5 Bool)
15 (declare-const A_3_5 Bool)
16 (declare-const A_1_6 Bool)
17 (declare-const A_2_6 Bool)
18 (declare-const A_3_6 Bool)
19 (declare-const A_1_7 Bool)
20 (declare-const A_2_7 Bool)
21 (declare-const A_3_7 Bool)
22 (declare-const A_1_8 Bool)
23 (declare-const A_2_8 Bool)
24 (declare-const A_3_8 Bool)
25 (declare-const A_4_1 Bool)
26 (declare-const A_5_1 Bool)
27 (declare-const A_4_2 Bool)
28 (declare-const A_5_2 Bool)
29 (declare-const A_4_3 Bool)
30 (declare-const A_5_3 Bool)
31 (declare-const A_4_4 Bool)
32 (declare-const A_5_4 Bool)
33 (declare-const A_4_5 Bool)
34 (declare-const A

131 (define-fun NOT_78e7d4380262ec16 () Bool (not AND_e72c6edfeaf78500))(define-fun AND_c73f24b8dd6af83e () Bool (and A_4_1 A_4_2 A_4_3))
132 (define-fun NOT_491cfd72123536fc () Bool (not AND_c73f24b8dd6af83e))(define-fun AND_33dbca13eb9caf1f () Bool (and A_5_1 A_5_2 A_5_3))
133 (define-fun NOT_1e12d603f6c77deb () Bool (not AND_33dbca13eb9caf1f))(define-fun AND_29788cb84c19aca0 () Bool (and A_1_2 A_1_3 A_1_4))
134 (define-fun NOT_cdf139b91870eeb6 () Bool (not AND_29788cb84c19aca0))(define-fun AND_db63f5cf4c4c2233 () Bool (and A_2_2 A_2_3 A_2_4))
135 (define-fun NOT_5a78e8fd3c944312 () Bool (not AND_db63f5cf4c4c2233))(define-fun AND_c640dcfdd2ada9ea () Bool (and A_3_2 A_3_3 A_3_4))
136 (define-fun NOT_c9e35a96ff9cb46e () Bool (not AND_c640dcfdd2ada9ea))(define-fun AND_2f57346120dbb07 () Bool (and A_4_2 A_4_3 A_4_4))
137 (define-fun NOT_7d33765f49b9c9c5 () Bool (not AND_2f57346120dbb07))(define-fun AND_e96c4a42e5e6abcf () Bool (and A_5_2 A_5_3 A_5_4))
138 (define-fun NOT_fcf31c28ee656c2 